In [2]:
import pandas as pd
import numpy as np
import math
import copy
import networkx as nx
from datetime import timedelta, datetime, date
from itertools import combinations

# 1 Import Data

This data processing part is the same as that of the file 'Directed_graph_train_number'.


We will focus on all the stops within 10km of the Zurich train station.

We calculated distance using this formula:

$$ DISTANCE = 2* arcsin\sqrt{sin^2\frac{a}{2}+cos(Lat1)*cos(Lat2)*sin^2\frac{b}{2}} * Earth Radius $$
$$ a = Lat1-Lat2, b = Lon1 - Lon2 $$

In [3]:
#Load geo data
geo = pd.read_csv('./data/BFKOORD_GEO', sep="%", header=None,error_bad_lines=False)
geo.columns = ['data','name']
geo.name = geo.name.apply(str.lstrip).apply(str.rstrip)

geo[['station_number','longtitude','latitude','height']] = geo.data.str.split(expand=True)#.apply(float)
geo.drop('data',axis=1,inplace=True)

# station in Zurich
zurich = geo[geo.station_number=="8503000"].reset_index(drop=True)

In [4]:
zurich

,name,station_number,longtitude,latitude,height
0,Zürich HB,8503000,8.540192,47.378177,408


In [5]:
# Define a function to compute the distance with the longtitude and the latitude
from math import sin, cos, sqrt, atan2, radians,asin
def compute_distance(point_1_lat, point_1_lon, point_2_lat=zurich.latitude, point_2_lon=zurich.longtitude):
    # approximate radius of earth in km
    R = 6378.137 # earth radius

    lat1 = radians(float(point_1_lat))
    lon1 = radians(float(point_1_lon))
    lat2 = radians(float(point_2_lat))
    lon2 = radians(float(point_2_lon))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))

    distance = R * c
    return np.round(distance,3) # return distance in kilometres

In [6]:
#Keep Zurich neighbourhood stations
distance = []
for log,lat in zip(geo.longtitude,geo.latitude):
    distance.append(compute_distance(lat,log))

geo['distance'] = distance
zurich_neigh_station = geo[geo.distance <= 10]

In [8]:
#generate a dataframe that record the distance between two stations in the zurich neighbourhood area.
distMap       = []
nodes_list    = list(zurich_neigh_station.name)
combiList     = list(combinations(nodes_list, 2))
count, length = 0, len(combiList)

for placeA, placeB in combiList:
    placeAdf = zurich_neigh_station[zurich_neigh_station['name']==placeA]
    placeBdf = zurich_neigh_station[zurich_neigh_station['name']==placeB]
    distance = compute_distance(placeAdf.latitude.values[0], 
                                placeAdf.longtitude.values[0], 
                                placeBdf.latitude.values[0], 
                                placeBdf.longtitude.values[0])
    count += 1
    if count % np.ceil(length/100) == 0:
        print('{}%'.format(count / np.ceil(length/100)))
    distMap.append([placeA, placeB, distance])
    distMap.append([placeB, placeA, distance])
distMapNewDf = pd.DataFrame(distMap, columns=['placeA', 'placeB', 'distance'])
distMapNewDf.to_csv('./data/distMapNew.csv')

In [9]:
#Load data on April 30th / April 29th / April 28th, 2018 in order to build a graph
df_18_04 = pd.read_csv("./data/grouped_201804.csv")
df_day = df_18_04[(df_18_04['date_of_trip'] == '30.04.2018')]
df_day = df_18_04[(df_18_04['date_of_trip'] == '30.04.2018')&(df_18_04['additional_trip'] == False)&\
                  (df_18_04['not_stop'] == False)]
df_day = df_day.reset_index(drop=True)
nan = np.nan
df_day['Timetable'] = df_day.Timetable.map(lambda x:eval(x))
df_day = df_day[['date_of_trip', 'identifies_of_trip', 'Timetable', 'train_number']]
all_station = []
for idx, row in df_day.iterrows():
    station = []
    for i in range(len(row['Timetable'])):
        station.append(row['Timetable'][i][0])
    all_station.append(station)
df_day['station_name'] = all_station
df_day.head()

,date_of_trip,identifies_of_trip,Timetable,train_number,station_name
0,30.04.2018,85:11:1507:002,"[[Zürich HB, 23400, 23593, 23940, 23984], [Zür...",1507,"[Zürich HB, Zürich Flughafen]"
1,30.04.2018,85:11:1509:003,"[[Zürich HB, 27000, 27113, 27540, 27582], [Zür...",1509,"[Zürich HB, Zürich Flughafen]"
2,30.04.2018,85:11:1510:003,"[[Zürich Flughafen, 25860, 25906, 25980, 26102...",1510,"[Zürich Flughafen, Zürich HB]"
3,30.04.2018,85:11:1511:003,"[[Zürich HB, 30600, 30690, 31140, 31223], [Zür...",1511,"[Zürich HB, Zürich Flughafen]"
4,30.04.2018,85:11:1512:003,"[[Zürich Flughafen, 29460, 29451, 29580, 29710...",1512,"[Zürich Flughafen, Zürich HB]"


# 2 Graph Construction

This part is used for creating a multiedge graph. The nodes are station names. An edge in the graph contains trip id, departure time from the last node and arrvial time at the next node. 

We connect a pair of nodes by an edge labeled as 'Walk' if the distance between them is less than a manually set maximum walking distance. The attributes 'arrToNext' and 'depFromLast' are left None when creating walking edges. These two values will be set automatically when searching for a valid path. We assume that one walks immediately to the next interchange station after his arrival at the end of the first trip, and the walking time is estimated by:
$\frac{distance}{walk\ speed}$



In [10]:
def create_graph(multi=True):
    
    '''
    This function is used to create a multiedge graph.
    nodes: station names
    edges: draw an edge between node A and node B when they are linked by train / bus / tram.
    Edge attributes: trip id, lines id, departure time from node A and arrival time at node B.
    '''
    if multi:
        G = nx.MultiDiGraph()
    else:
        G = nx.DiGraph()
        
    # Add nodes to graph
    nodes_list = list(set(zurich_neigh_station['name'].unique()))
    G.add_nodes_from(nodes_list)
    # Add directed edges
    for idx, row in df_day.iterrows():
        trip_id = row['identifies_of_trip']
        for j in range(len(row['Timetable'])-1):
            
            # Assume that nodes are ordered by default
            node1 = row['Timetable'][j][0]
            node2 = row['Timetable'][j+1][0]
            depFromLast = row['Timetable'][j][3]
            arrToNext   = row['Timetable'][j+1][1]
            
            # Only add the edges having the aimed time and not None
            if np.isnan(depFromLast) == False and np.isnan(arrToNext) == False and depFromLast <= arrToNext:
                G.add_edge(node1, node2, 
                           trip_id=trip_id, trainId=row['train_number'], 
                           arrToNext=datetime(2017,9,13,0,0) + timedelta(seconds=arrToNext), \
                           depFromLast=datetime(2017,9,13,0,0) + timedelta(seconds=depFromLast))
                
    return G


In [11]:
def add_walk_edge(G, maxWalk=0.15): 
    
    '''
    This function is used to add edges that signify "Walk" on graph.
    maxWalk: the maximum walking distance.
    '''
    distMap = distMapNewDf[distMapNewDf.distance < maxWalk]
    for idx, row in distMap.iterrows():
        G.add_edge(row['placeA'], 
                   row['placeB'], 
                   trip_id = 'Walk',
                   distance = row['distance'],
                   arrToNext   = None,
                   depFromLast = None)
    return G


# 3 Path Searching

In this part, we first define a function to search for all directed journeys. Then we search for all valid paths on the graph. To reduce search time, we define many constraints, for example, start time of the journey, end time of the journey, number of stations, number of transfers, waiting time at a station, number of walks, no consecutive walks.

In [12]:
def direct_trip(source, target, start_time='12:00', interval=timedelta(minutes=30)):
    
    '''
    This function is used to find direct trips.

    Input: 
    source: the orgin of the journey
    target: the destination of the journey
    start_time: departure time
    interval: Find journeys that start within 30 minutes (default) from the departure time
    '''
    result = []
    start_time = datetime.strptime('2017/09/13 '+start_time, '%Y/%m/%d %H:%M')
    for idx, row in df_day.iterrows():
        if (source in row['station_name'])&(target in row['station_name']):
            source_idx = row['station_name'].index(source)
            target_idx = row['station_name'].index(target)
            if source_idx <target_idx:
                dep_time = row['Timetable'][source_idx][3]
                dep_time = datetime(2017,9,13,0,0) + timedelta(seconds=dep_time)
                arr_time = row['Timetable'][target_idx][1]
                arr_time = datetime(2017,9,13,0,0) + timedelta(seconds=arr_time)
                if (dep_time>=start_time)&(dep_time<=start_time+interval):
                    station_list = row['station_name'][source_idx:target_idx+1]
                    value = {'trip_id': row['identifies_of_trip'],
                             'arrToNext': arr_time,
                             'depFromLast': dep_time}
                    result.append((station_list, value))
        result = sorted(result,key=lambda x: x[-1]['arrToNext'])
    return result
  

In [13]:
def set_stopnum(source, target):

    '''
    Set the number of stops in a journey
    '''
    stopnum = 15
    if direct_trip(source, target):
        candidate = max([len(i[0]) for i in direct_trip(source, target)])
        if candidate < 10:
            stopnum = candidate
    elif float(distMapNewDf[(distMapNewDf['placeA'] == source)&(distMapNewDf['placeB'] == target)].distance) <0.5:
        stopnum = 2
        
    return stopnum

We employ DFS algorithm in our multiedge graph. Since schedule information has already been embedded in the graph edges, we can recommend reasonable routes containing trip ids and station names directly. The start time of the next trip should be later than the end time of the last trip, so the client can catch the next train or bus in the interchange sation. The waiting time is set to be less than 10 minutes at a station.

In [14]:
def all_simple_paths_multigraph(last_arr_time, G, source, target, StartTime, interval=timedelta(minutes=10), StopN=None, PathN=10, TripN=10):
    
    """
    Compute all paths of a multigraph from source to target using DFS.
    Visited: [A              , B              , C               ...]
    Stack  : [Next Layer of A, Next Layer of B, Next Layer of C ...]
    
    Input:
        last_arr_time: Set the lastest arrival time of the journey
        G:               Graph
        source:          Source vertex
        target:          Target Vertex
        StartTime:       Start time, should be in format (%H:%M)
        interval:        Longest time the user can wait
        StopN:           Largest number of vertex in path (at least 2, at most len(G))
        TripN:           The maximum number of trip ids in a journey
        PathN:           Top N fastest path will be returned
        maxWalk:         Max length (/km) limit of walk
    """ 
    
    def reset_walk_edge_attr(val):
        distance = val['distance']
        walkTime = timedelta(minutes=math.ceil(distance/walkSpeed*60))

        if type(lastchildAttr) == type(None):
            val['depFromLast'] = StartTime
            val['arrToNext']   = StartTime + walkTime

        else:
            val['depFromLast'] = lastchildAttr[2]['arrToNext']
            val['arrToNext']   = lastchildAttr[2]['arrToNext'] + walkTime
        return val
    
    def set_time_constrints(nodeAttr):
        # Reset the atttibutes of walk edge
        if nodeAttr[2]['trip_id'] == 'Walk':
            reset_walk_edge_attr(nodeAttr[2])
                
        # Last node is source       
        if type(lastchildAttr) == type(None):
            if nodeAttr[2]['depFromLast'] + timedelta(seconds=1) <= StartTime or nodeAttr[2]['depFromLast'] + timedelta(seconds=1) >= StartTime + interval:
                return False, nodeAttr
        else:
            # Drop edges with one side unknown
            if type(nodeAttr[2]['depFromLast']) == type(None) or lastchildAttr[2]['arrToNext'] == type(None):
                return False, nodeAttr
            # Drop edges with wrong order
            elif nodeAttr[2]['depFromLast'] + timedelta(seconds=1) <= lastchildAttr[2]['arrToNext'] or nodeAttr[2]['depFromLast'] + timedelta(seconds=1) >= lastchildAttr[2]['arrToNext'] + interval:
                return False, nodeAttr
        return True, nodeAttr
    
    def get_path_with_attribute(G, node):
        # Final next layer nodes
        pathWithAttr  = []
        G_dict        = G[node]
        # Add bus edges
        for vertex, attribute in G_dict.items():
            # Set non-loop constraint
            if vertex not in visited:
                for key, value in attribute.items():
                    
                    flag, nodeAttr = set_time_constrints((node, vertex, value))
                    if flag:
                        pathWithAttr.append(nodeAttr)
        return pathWithAttr
    
    def update_shortest_path(shortest_path_, last_arr_time_, childAttr):
        childAttrCopy = copy.deepcopy(childAttr)
        path    = (visitedWithAttr + [childAttrCopy])[1:]
        tripnum = len(list(set([i[2]['trip_id'] for i in path if i[2]['trip_id'] != 'Walk'])))
        walknum = np.sum([1 for i in path if i[2]['trip_id'] == 'Walk'])
        trip_id_list = [i[2]['trip_id'] for i in path]
        consecutive_walk = \
        any(trip_id_list[i] == trip_id_list[i+1] for i in range(len(trip_id_list)-1) if trip_id_list[i] == 'Walk')
        
        # Sort top N fastest paths, the number of walks is no more than 2 and one cannot walk consecutively
        
        if (tripnum <= TripN)&(walknum <=2)&(consecutive_walk == False):
            if len(shortest_path_) < PathN:
                shortest_path_.append(path)
                shortest_path_ = sorted(shortest_path_, key=lambda x: x[-1][2]['arrToNext'])
                last_arr_time_ = shortest_path_[0][-1][2]['arrToNext']
            else:
                if childAttr[2]['arrToNext'] < last_arr_time_:
                    shortest_path_.append(path)
                    shortest_path_ = sorted(shortest_path_, key=lambda x: x[-1][2]['arrToNext'])[:PathN]
                    last_arr_time_ = shortest_path_[0][-1][2]['arrToNext']
        return shortest_path_, last_arr_time_
    
    # ---------------------------------------- Start ---------------------------------------- #
    # Set the number of vertex in path (Depth of the path)
    if StopN is None:
        cutoff = len(G) - 1
    elif StopN < 2:
        return
    else:
        cutoff = StopN - 1
       
    # Set walk speed
    walkSpeed = 5
    
    # Maintain the list of first 'PathN' shortest path
    shortest_path = []
    
    # The vertices that have been visited
    visited, visitedWithAttr = [source], [None]
    lastchild, lastchildAttr = source,    None
    
    # Stack used to store the depth tree
    stack = [(path for path in get_path_with_attribute(G, source))]

    # Run until the tree is totally searched
    while stack:
        lastchild     = visited[-1]
        lastchildAttr = visitedWithAttr[-1]
        
        # Generator: Next layer of last vertex in visited list
        children  = stack[-1]
        # *It will apply to the same generator as used last time
        childAttr = next(children, None)
        child = childAttr[1] if type(childAttr)!=type(None) else None
        
        # All the child have been searched  
        if child is None:
            stack.pop()
            visited.pop()
            visitedWithAttr.pop()
            
        else:
            # If the child is not None, and visited nodes < cutoff
            if len(visited) < cutoff:
                # If the target is reached
                if child == target:
                    if childAttr[2]['arrToNext'] <= last_arr_time:
                        shortest_path, last_arr_time = update_shortest_path(shortest_path, last_arr_time, childAttr)
                    
                # Set constrints to depth
                elif child not in visited:
                    tripnum = len(list(set([i[2]['trip_id'] for i in visitedWithAttr[1:] if i[2]['trip_id'] != 'Walk'])))
                    # If the child 'arrToNext' larger than last_arr_time, its children will not be searched
                    if childAttr[2]['arrToNext'] >= last_arr_time or tripnum > TripN - 1:
                        pass
                    else:
                        visited.append(child)
                        visitedWithAttr.append(copy.deepcopy(childAttr))
                        lastchildAttr = visitedWithAttr[-1]
                        lastchild     = visited[-1]
                        stack.append((path for path in get_path_with_attribute(G, child)))
                        
            else: 
                targetDict = G.get_edge_data(lastchild, target)
                if type(targetDict)!=type(None):
                    for _, val in targetDict.items():
                        
                        if val['trip_id'] == 'Walk':
                            reset_walk_edge_attr(val)
                        if type(lastchildAttr) != type(None) \
                            and lastchildAttr[2]['arrToNext'] < val['depFromLast'] + timedelta(seconds=1) \
                            and lastchildAttr[2]['arrToNext'] + interval > val['depFromLast'] + timedelta(seconds=1):
                            shortest_path, last_arr_time = update_shortest_path(shortest_path, last_arr_time, (lastchild, target, val))
                           
                        # Only 2 stops in path
                        elif type(lastchildAttr) == type(None)\
                             and val['depFromLast'] <= val['arrToNext'] \
                             and val['depFromLast'] >= StartTime and val['depFromLast'] <= StartTime + interval:
                             shortest_path, last_arr_time = update_shortest_path(shortest_path, last_arr_time, (lastchild, target, val))
                stack.pop()
                visited.pop()
                visitedWithAttr.pop()
    return shortest_path

def get_paths(G, source, target, StartTime, interval=timedelta(minutes=10), stopnum=None, TripN=4, PathN=20, \
             last_arr_time=None):
    
    '''
    Get recommmended paths from the graph
    '''
    StartTime = datetime.strptime('2017/09/13 '+StartTime, '%Y/%m/%d %H:%M')
    if last_arr_time == None:
        last_arr_time = StartTime + timedelta(minutes=40)
    else: 
        last_arr_time = datetime.strptime('2017/09/13 '+last_arr_time, '%Y/%m/%d %H:%M')
    paths = []
    count = 0
    while len(paths) == 0:
        paths = all_simple_paths_multigraph(last_arr_time, G, source, target, StartTime, interval=interval,\
                                            StopN=stopnum, TripN=TripN, PathN=PathN)
        
        last_arr_time = last_arr_time + timedelta(minutes=30)
        count +=1
        if count == 5:
            break
    return paths


In [15]:
def get_all_paths(G, source, target, StartTime, interval=timedelta(minutes=10), stopnum=None, TripN=4, PathN=20, \
                  last_arr_time=None):
    
    '''
    Concatenate path solutions, combining directed journeys and others that need transfering.
    '''
    paths_direct = direct_trip(source, target, StartTime)  
    result_direct = []
    for path in paths_direct:
        station_list1 = [path[0][0], path[0][-1]]
        trip_id_list1 = [path[1]['trip_id']]
        trip_id_list1 = [[i] for i in trip_id_list1]
        result_direct.append([station_list1, trip_id_list1])

    paths_graph = get_paths(G, source, target, StartTime, interval=interval, stopnum=stopnum, TripN=TripN, PathN=PathN, \
                            last_arr_time=last_arr_time)
    result_graph = []
    for path in paths_graph:
        station_list2 = [i[1] for i in path]
        trip_id_list2 = [i[2]['trip_id'] for i in path]
        change_index = np.where(np.array(trip_id_list2[:-1]) != np.array(trip_id_list2[1:]))[0]
        if len(change_index)>0:
            station_list2 = [station_list2[i] for i in change_index]
            station_list2 = [path[0][0]] + station_list2 + [path[-1][1]]
            trip_id_list2 = [trip_id_list2[i] for i in change_index] + [trip_id_list2[-1]]
            trip_id_list2 = [[i] for i in trip_id_list2]
            result_graph.append([station_list2, trip_id_list2])
        
    return result_direct + result_graph

# 4 Path Recommendation

In [16]:
#load distance dataframe and create graphs with maximum walking distance of 200m and 400m.
distMapNewDf = pd.read_csv('./new_data/distMapNew.csv', index_col=0)
G_multidi_200 = create_graph(multi=True)
G_multidi_200 = add_walk_edge(G_multidi_200, 0.2)
G_multidi_400 = create_graph(multi=True)
G_multidi_400 = add_walk_edge(G_multidi_400, 0.4)

Here are three route planning examples. Different paths are aranged in the order of arrival time (earliest - latest). 

'Birmensdorf ZH, Bahnhof' - 'Dietikon, Birmensdorferstrasse':

Route 1: First walk to the station 'Birmensdorf ZH, Bahnhof' and then take the trip id '85:849:310448-32301-1' to the destination, which is in line with the solution in the SBB Mobile App.

We also recommend other routes that needs transferring 

In [26]:
#Generate Recommended routes
source = 'Birmensdorf ZH'
target = 'Dietikon, Birmensdorferstrasse'
StartTime = '12:30' 
stopnum = set_stopnum(source, target)
final_result = get_all_paths(G_multidi_400, source, target, StartTime, stopnum=stopnum)
final_result = [[trip] for trip in final_result]
final_result

[[[['Birmensdorf ZH',
    'Birmensdorf ZH, Bahnhof',
    'Dietikon, Birmensdorferstrasse'],
   [['Walk'], ['85:849:310448-32301-1']]]],
 [[['Birmensdorf ZH',
    'Birmensdorf ZH, Bahnhof',
    'Urdorf, Herweg',
    'Dietikon, Birmensdorferstrasse'],
   [['Walk'], ['85:849:310448-32301-1'], ['85:849:309707-32301-1']]]],
 [[['Birmensdorf ZH',
    'Zürich Altstetten',
    'Glanzenberg',
    'Dietikon, Birmensdorferstrasse'],
   [['85:11:18547:001'], ['85:11:19246:001'], ['Walk']]]],
 [[['Birmensdorf ZH',
    'Urdorf Weihermatt',
    'Urdorf, Uitikonerstrasse',
    'Glanzenberg, Bahnhof',
    'Dietikon, Birmensdorferstrasse'],
   [['85:11:18547:001'], ['Walk'], ['85:849:301661-32301-1'], ['Walk']]]],
 [[['Birmensdorf ZH',
    'Urdorf Weihermatt',
    'Urdorf Weihermatt, Bahnhof',
    'Glanzenberg, Bahnhof',
    'Dietikon, Birmensdorferstrasse'],
   [['85:11:18547:001'], ['Walk'], ['85:849:301661-32301-1'], ['Walk']]]],
 [[['Birmensdorf ZH',
    'Birmensdorf ZH, Bahnhof',
    'Urdorf, Spitz

'Zürich HB' - 'Thalwil, Sonnenberg'
The first route match up to the solution in the SBB Mobile App.

In [15]:
#Generate Recommedned routes
source2 = 'Zürich HB'
target2 = 'Thalwil, Sonnenberg'
StartTime2 = '15:36' 
stopnum2 = set_stopnum(source2, target2)
final_result2 = get_all_paths(G_multidi_200, source2, target2, StartTime2, \
                              interval=timedelta(minutes=5), stopnum=stopnum2, \
                              TripN=3, PathN=20, last_arr_time='16:05')
final_result2 = [[trip] for trip in final_result2]
final_result2

[[[['Zürich HB', 'Thalwil', 'Thalwil, Zentrum', 'Thalwil, Sonnenberg'],
   [['85:11:18859:001'], ['Walk'], ['85:807:534553-58131-1']]]],
 [[['Zürich HB', 'Thalwil', 'Thalwil, Post', 'Thalwil, Sonnenberg'],
   [['85:11:18859:001'], ['Walk'], ['85:807:534553-58131-1']]]]]

'Zürich Oerlikon' - 'Opfikon, Bahnhof':
The first route is quickest and is also what the official app recommends.

In [20]:
#Generate Recommended routes
source3 = 'Zürich Oerlikon'
target3 = 'Opfikon, Bahnhof'
StartTime = '12:30' 
stopnum3 = set_stopnum(source3, target3)
final_result3 = get_all_paths(G_multidi_400, source3, target3, StartTime, stopnum=5)
final_result3 = [[trip] for trip in final_result3]
final_result3


[[[['Zürich Oerlikon', 'Glattbrugg', 'Opfikon, Bahnhof'],
   [['85:11:19544:001'], ['Walk']]]],
 [[['Zürich Oerlikon',
    'Glattbrugg',
    'Glattbrugg, Frohbühlstrasse',
    'Glattbrugg, Post',
    'Opfikon, Bahnhof'],
   [['85:11:19544:001'], ['Walk'], ['85:773:527074-01733-1'], ['Walk']]]],
 [[['Zürich Oerlikon',
    'Glattbrugg',
    'Glattbrugg, Post',
    'Glattbrugg, Frohbühlstrasse',
    'Opfikon, Bahnhof'],
   [['85:11:19544:001'], ['Walk'], ['85:773:526879-01733-1'], ['Walk']]]]]